In [5]:
import sys
# sys.path.insert(0, "../../")

import inspect

In [4]:
from collections import OrderedDict    # For recording the model specification 
import urbansim_templates
import pandas as pd                    # For file input/output
import numpy as np                     # For vectorized math operations
import statsmodels.tools.numdiff as numdiff       # For numeric hessian
import scipy.linalg                    # For matrix inversion

import pylogit as pl                   # For choice model estimation
from pylogit import nested_logit as nl # For nested logit convenience funcs
import orca
import os; os.chdir('../')

from urbansim_templates import modelmanager as mm
from urbansim_templates.models import LargeMultinomialLogitStep
from urbansim_templates.models import SmallMultinomialLogitStep

from scripts import datasources, models, variables, utils

In [11]:
input_file_format = 'csv'
input_data_dir = '/home/data/spring_2019/base/'

formattable_fname_dict = {
    'parcels': 'parcels.{0}',
    'buildings': 'buildings.{0}',
    'jobs': 'jobs.{0}',
    'establishments': 'establishments.{0}',
    'households': 'households.{0}',
    'persons': 'persons.{0}',
    'rentals': 'craigslist.{0}',
    'units': 'units.{0}',
    'mtc_skims': 'mtc_skims.{0}',
    'beam_skims_raw': '30.skims-smart-23April2019-baseline.csv.gz',
    'beam_skims_imputed': 'beam_skims_imputed.{0}',
    # the following nodes and edges .csv's will be phased out and
    # replaced by travel model skims entirely
    'drive_nodes': 'drive_nodes.{0}',
    'drive_edges': 'drive_edges.{0}',
    'drive_access_vars': 'drive_net_vars.{0}',
    'walk_nodes': 'walk_nodes.{0}',
    'walk_edges': 'walk_edges.{0}',
    'walk_access_vars': 'walk_net_vars.{0}',
    'zones': 'zones.{0}',
    'zone_access_vars': 'zones_w_access_vars.{0}',
}

def format_fname_dict(formattable_fname_dict, format='csv'):
    formatted_dict = {
        k: v.format('csv')
        for k, v in formattable_fname_dict.items()}
    return formatted_dict

input_fnames = format_fname_dict(
            formattable_fname_dict, input_file_format)

In [12]:
orca.add_injectable('input_file_format', input_file_format)
orca.add_injectable('input_data_dir', input_data_dir)
orca.add_injectable('input_fnames', input_fnames)
orca.add_injectable('store', None)

In [13]:
orca.run(['initialize_network_small']) 
orca.run([ 'initialize_network_walk']) 
orca.run(['initialize_imputed_skims']) 

Running step 'initialize_network_small'
Time to execute step 'initialize_network_small': 0.00 s
Total time to execute iteration 1 with iteration value None: 0.00 s
Running step 'initialize_network_walk'
Time to execute step 'initialize_network_walk': 0.00 s
Total time to execute iteration 1 with iteration value None: 0.00 s
Running step 'initialize_imputed_skims'


/home/juan/anaconda3/envs/template-env/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


No imputed skims found. Creating them now.
Time to execute step 'initialize_imputed_skims': 90.30 s
Total time to execute iteration 1 with iteration value None: 90.30 s


In [15]:
orca.list_tables()

['parcels',
 'buildings',
 'jobs',
 'establishments',
 'households',
 'persons',
 'rentals',
 'units',
 'zones',
 'access_indicators_ampeak',
 'mtc_skims',
 'beam_skims_raw',
 'beam_skims_imputed',
 'drive_nodes',
 'drive_edges',
 'walk_nodes',
 'walk_edges',
 'schools',
 'public_schools_50',
 'private_schools_100',
 'students',
 'closest_schools',
 'long_format']

In [18]:
beam_skims = orca.get_table('beam_skims_imputed').to_frame()
trips = pd.read_csv('/home/emma/activitysynth/activitysynth/data/HWtrips_042519.csv')

In [17]:
trips

,Unnamed: 0,HHPER,index,Unnamed: 0.1,HHPERTRIP,origin,home_dwell,home_ST,HW_trip_ST,HW_trip_ET,...,avg_income_gen_tt_CAR_AM_30,avg_income_gen_tt_WALK_TRANSIT_AM_30,avg_income_gen_tt_DRIVE_TRANSIT_AM_30,avg_income_gen_tt_BIKE_AM_30,avg_income_gen_tt_WALK_AM_30,gen_tt_CAR,gen_tt_BIKE,gen_tt_WALK,gen_tt_WALK_TRANSIT,gen_tt_DRIVE_TRANSIT
0,3992,30202193,9223,9547,3.020219e+09,home,14.583333,7.166667,21.750000,22.000000,...,282587.520824,280611.766021,284329.195532,279697.715188,336615.636364,3.276973,13.390643,51.472567,8.924065,5.991765
1,3203,25893411,7394,7663,2.589341e+09,home,13.933333,17.783333,7.716667,8.766667,...,227631.760137,229196.431064,227579.411547,235577.940921,220483.941379,9.487323,36.093055,157.685268,23.904427,15.866721
2,3155,25771411,7301,7560,2.577141e+09,home,13.433333,18.483333,7.916667,8.116667,...,374389.340936,382482.289292,377578.207537,379976.395742,449787.133518,8.799580,33.476644,146.254552,22.171578,14.716531
3,3957,30177401,9127,9449,3.017740e+09,home,11.233333,18.316667,5.550000,5.983333,...,277770.360660,279733.163534,277472.519810,284927.461754,322432.749072,33.164938,135.521383,520.933413,90.316921,60.640280
4,339,13427531,829,860,1.342753e+09,home,13.083333,17.666667,6.750000,7.500000,...,188707.408085,181761.139449,185527.602001,169262.064960,161212.310178,81.733333,200.390249,875.475644,132.718444,88.092741
5,3539,29884341,8159,8445,2.988434e+09,home,14.416667,18.250000,8.666667,9.583333,...,229780.908603,222218.780653,230351.852368,215933.295701,256087.760631,25.710998,97.813520,427.332939,64.781885,47.683333
6,2727,22328771,6301,6530,2.232877e+09,home,15.133333,18.916667,10.050000,10.350000,...,305744.662552,304488.136927,302174.040869,294704.167945,301971.732203,8.480607,34.654177,133.207899,23.094943,15.506328
7,4037,30232691,9330,9656,3.023269e+09,home,13.916667,19.166667,9.083333,9.500000,...,199653.419347,201186.832640,199730.691410,199772.770627,192373.391752,18.066667,56.487643,246.786238,45.472222,24.832302
8,2924,24956501,6759,7006,2.495650e+09,home,12.133333,18.116667,6.250000,7.416667,...,154228.506037,144633.833233,155548.195917,140399.092433,150271.713702,83.869345,319.067970,1393.961219,211.318687,140.264169
9,3988,30199971,9212,9536,3.019997e+09,home,20.333333,11.750000,8.083333,8.416667,...,259758.074232,279702.236827,267929.470619,286827.496177,268796.524723,3.473983,13.216230,57.739773,8.753108,5.809933


In [34]:
beam_skims = orca.get_table('beam_skims').to_frame().reset_index(level=['from_zone_id', 'to_zone_id'])
trips = pd.read_csv('/home/emma/activitysynth/activitysynth/data/HWtrips_042519.csv')

In [19]:
cost_variables = ['from_zone_id', 'to_zone_id','gen_cost_BIKE', 'gen_cost_CAR',
       'gen_cost_DRIVE_TRANSIT', 'gen_cost_RIDE_HAIL',
       'gen_cost_RIDE_HAIL_POOLED', 'gen_cost_RIDE_HAIL_TRANSIT',
       'gen_cost_WALK', 'gen_cost_WALK_TRANSIT']

trips_1 = trips.merge(beam_skims.loc[:,cost_variables], 
                      how = 'left', 
                      left_on = ['from_zone_id', 'to_zone_id'], 
                      right_on = ['from_zone_id', 'to_zone_id'])

/home/juan/anaconda3/envs/template-env/lib/python3.6/site-packages/pandas/core/indexing.py:979: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_nested_tuple(tup)
/home/juan/anaconda3/envs/template-env/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: 'from_zone_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  if __name__ == '__main__':
/home/juan/anaconda3/envs/template-env/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: 'to_zone_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  if __name__ == '__main__':


In [20]:
trips_1['DRIVE'] = 1
trips_1['SHARED'] = 1
trips_1['WALK_TRANSIT'] = 1
trips_1['DRIVE_TRANSIT'] = 1
trips_1['BIKE'] = 1
trips_1['WALK'] = 1

In [21]:
trips["custom_id"] = np.arange(trips.shape[0], dtype=int) + 1

In [22]:
pd.set_option('display.max_columns', None)
trips_1.head()

,Unnamed: 0,HHPER,index,Unnamed: 0.1,HHPERTRIP,origin,home_dwell,home_ST,HW_trip_ST,HW_trip_ET,HW_TT,Hzone,Wzone,Hxcord,Hycord,Wxcord,Wycord,HW_mode,work_dwell,work_ST,WH_trip_ST,WH_trip_ET,WH_TT,WH_mode,modeHW,TOD,dwell_work,SAMPN,PERNO,GEND,AGE,HISP,RACE1,RACE2,RACE3,RACE4,hours,EDUCA,INDUS,HHVEH,OWN,INCOM,HHSIZ,minority,black,native,asian,PI,hh_inc_less10k,hh_inc_10kless25k,hh_inc_25kless35k,hh_inc_35kless50k,hh_inc_50kless75k,hh_inc_100kless150k,hh_inc_150kless200k,hh_inc_200kless250k,hh_inc_less50k,hh_inc_less75k,hh_inc_75kless100k,hh_inc_150kplus,hh_inc_150kless250k,hh_inc_250kplus,lessGED,GED,somebach,Assoc,Bach,lessGED_GED,lessbach,age_16less25,age_25less40,age_40less50,age_50less60,female,tenure_2,noveh,1veh,2veh,3plusveh,hh_size_1per,hh_size_2per,hh_size_3per,hh_size_4plusper,sector_constr,sector_mfg,sector_retail,sector_transport,info,finance,scitech,sector_edu_serv,sector_healthcare,sector_oth_serv,sector_gov,TOD_3to6,TOD_6to9,TOD_9to1530,TOD_1530to1830,TOD_1830up,TOD_9up,TOD_1530up,from_zone_id,to_zone_id,gen_tt_CAR_AM,gen_tt_BIKE_AM,gen_tt_DRIVE_TRANSIT_AM,gen_tt_WALK_AM,gen_tt_WALK_TRANSIT_AM,gen_tt_CAR_PM,gen_tt_BIKE_PM,gen_tt_DRIVE_TRANSIT_PM,gen_tt_WALK_PM,gen_tt_WALK_TRANSIT_PM,gen_tt_CAR_OffPeak,gen_tt_BIKE_OffPeak,gen_tt_DRIVE_TRANSIT_OffPeak,gen_tt_WALK_OffPeak,gen_tt_WALK_TRANSIT_OffPeak,total_jobs_gen_tt_CAR_AM_30,total_jobs_gen_tt_WALK_TRANSIT_AM_30,total_jobs_gen_tt_DRIVE_TRANSIT_AM_30,total_jobs_gen_tt_BIKE_AM_30,total_jobs_gen_tt_WALK_AM_30,sum_residential_units_gen_tt_CAR_AM_30,sum_residential_units_gen_tt_WALK_TRANSIT_AM_30,sum_residential_units_gen_tt_DRIVE_TRANSIT_AM_30,sum_residential_units_gen_tt_BIKE_AM_30,sum_residential_units_gen_tt_WALK_AM_30,sum_persons_gen_tt_CAR_AM_30,sum_persons_gen_tt_WALK_TRANSIT_AM_30,sum_persons_gen_tt_DRIVE_TRANSIT_AM_30,sum_persons_gen_tt_BIKE_AM_30,sum_persons_gen_tt_WALK_AM_30,avg_income_gen_tt_CAR_AM_30,avg_income_gen_tt_WALK_TRANSIT_AM_30,avg_income_gen_tt_DRIVE_TRANSIT_AM_30,avg_income_gen_tt_BIKE_AM_30,avg_income_gen_tt_WALK_AM_30,gen_tt_CAR,gen_tt_BIKE,gen_tt_WALK,gen_tt_WALK_TRANSIT,gen_tt_DRIVE_TRANSIT,gen_cost_BIKE,gen_cost_CAR,gen_cost_DRIVE_TRANSIT,gen_cost_RIDE_HAIL,gen_cost_RIDE_HAIL_POOLED,gen_cost_RIDE_HAIL_TRANSIT,gen_cost_WALK,gen_cost_WALK_TRANSIT,DRIVE,SHARED,WALK_TRANSIT,DRIVE_TRANSIT,BIKE,WALK
0,3992,30202193,9223,9547,3.020219e+09,home,14.583333,7.166667,21.750000,22.000000,0.250000,525.0,523.0,-121.896879,37.280818,-121.918797,37.264669,drive_alone,9.000000,22.000000,7.000000,7.166667,0.166667,drive_alone,0,4,4,3020219,3,1.0,21.0,0,1.0,NaN,NaN,NaN,30.0,2.0,44.0,1,1.0,6.0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0.0,0.0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,525,523,3.509251,13.350405,5.868917,58.325964,8.841972,4.349597,13.422655,6.335453,52.656144,8.967193,3.276973,13.390643,5.991765,51.472567,8.924065,383050.0,186953.0,329860.0,75337.0,826.0,378239.0,233326.0,387325.0,106794.0,3272.0,998095.0,579609.0,1026804.0,253673.0,8064.0,282587.520824,280611.766021,284329.195532,279697.715188,336615.636364,3.276973,13.390643,51.472567,8.924065,5.991765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,1,1,1
1,3203,25893411,7394,7663,2.589341e+09,home,13.933333,17.783333,7.716667,8.766667,1.050000,183.0,69.0,-122.496138,37.736289,-122.450262,37.776308,drive_alone,8.033333,8.766667,16.800000,17.783333,0.983333,drive_alone,0,1,3,2589341,1,2.0,37.0,0,4.0,NaN,NaN,NaN,40.0,6.0,52.0,1,1.0,8.0,5,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,1.0,0.0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,183,69,9.487323,36.093055,15.866721,157.685268,23.904427,11.759213,36.288384,17.128008,142.356813,24.242964,14.166667,36.201839,16.198843,139.156989,24.126367,406461.0,104443.0,374588.0,31419.0,1541.0,436772.0,261837.0,420902.0,118796.0,10802.0,915147.0,599577.0,895780.0,273038.0,27471.0,227631.760137,229196.431064,227579.411547,235577.940921,220483.941379,9.487323,36.093055,157.685268,23.904427,15.866721,NaN,NaN,NaN,NaN,NaN,

In [23]:
trips_1.head()

,Unnamed: 0,HHPER,index,Unnamed: 0.1,HHPERTRIP,origin,home_dwell,home_ST,HW_trip_ST,HW_trip_ET,HW_TT,Hzone,Wzone,Hxcord,Hycord,Wxcord,Wycord,HW_mode,work_dwell,work_ST,WH_trip_ST,WH_trip_ET,WH_TT,WH_mode,modeHW,TOD,dwell_work,SAMPN,PERNO,GEND,AGE,HISP,RACE1,RACE2,RACE3,RACE4,hours,EDUCA,INDUS,HHVEH,OWN,INCOM,HHSIZ,minority,black,native,asian,PI,hh_inc_less10k,hh_inc_10kless25k,hh_inc_25kless35k,hh_inc_35kless50k,hh_inc_50kless75k,hh_inc_100kless150k,hh_inc_150kless200k,hh_inc_200kless250k,hh_inc_less50k,hh_inc_less75k,hh_inc_75kless100k,hh_inc_150kplus,hh_inc_150kless250k,hh_inc_250kplus,lessGED,GED,somebach,Assoc,Bach,lessGED_GED,lessbach,age_16less25,age_25less40,age_40less50,age_50less60,female,tenure_2,noveh,1veh,2veh,3plusveh,hh_size_1per,hh_size_2per,hh_size_3per,hh_size_4plusper,sector_constr,sector_mfg,sector_retail,sector_transport,info,finance,scitech,sector_edu_serv,sector_healthcare,sector_oth_serv,sector_gov,TOD_3to6,TOD_6to9,TOD_9to1530,TOD_1530to1830,TOD_1830up,TOD_9up,TOD_1530up,from_zone_id,to_zone_id,gen_tt_CAR_AM,gen_tt_BIKE_AM,gen_tt_DRIVE_TRANSIT_AM,gen_tt_WALK_AM,gen_tt_WALK_TRANSIT_AM,gen_tt_CAR_PM,gen_tt_BIKE_PM,gen_tt_DRIVE_TRANSIT_PM,gen_tt_WALK_PM,gen_tt_WALK_TRANSIT_PM,gen_tt_CAR_OffPeak,gen_tt_BIKE_OffPeak,gen_tt_DRIVE_TRANSIT_OffPeak,gen_tt_WALK_OffPeak,gen_tt_WALK_TRANSIT_OffPeak,total_jobs_gen_tt_CAR_AM_30,total_jobs_gen_tt_WALK_TRANSIT_AM_30,total_jobs_gen_tt_DRIVE_TRANSIT_AM_30,total_jobs_gen_tt_BIKE_AM_30,total_jobs_gen_tt_WALK_AM_30,sum_residential_units_gen_tt_CAR_AM_30,sum_residential_units_gen_tt_WALK_TRANSIT_AM_30,sum_residential_units_gen_tt_DRIVE_TRANSIT_AM_30,sum_residential_units_gen_tt_BIKE_AM_30,sum_residential_units_gen_tt_WALK_AM_30,sum_persons_gen_tt_CAR_AM_30,sum_persons_gen_tt_WALK_TRANSIT_AM_30,sum_persons_gen_tt_DRIVE_TRANSIT_AM_30,sum_persons_gen_tt_BIKE_AM_30,sum_persons_gen_tt_WALK_AM_30,avg_income_gen_tt_CAR_AM_30,avg_income_gen_tt_WALK_TRANSIT_AM_30,avg_income_gen_tt_DRIVE_TRANSIT_AM_30,avg_income_gen_tt_BIKE_AM_30,avg_income_gen_tt_WALK_AM_30,gen_tt_CAR,gen_tt_BIKE,gen_tt_WALK,gen_tt_WALK_TRANSIT,gen_tt_DRIVE_TRANSIT,gen_cost_BIKE,gen_cost_CAR,gen_cost_DRIVE_TRANSIT,gen_cost_RIDE_HAIL,gen_cost_RIDE_HAIL_POOLED,gen_cost_RIDE_HAIL_TRANSIT,gen_cost_WALK,gen_cost_WALK_TRANSIT,DRIVE,SHARED,WALK_TRANSIT,DRIVE_TRANSIT,BIKE,WALK
0,3992,30202193,9223,9547,3.020219e+09,home,14.583333,7.166667,21.750000,22.000000,0.250000,525.0,523.0,-121.896879,37.280818,-121.918797,37.264669,drive_alone,9.000000,22.000000,7.000000,7.166667,0.166667,drive_alone,0,4,4,3020219,3,1.0,21.0,0,1.0,NaN,NaN,NaN,30.0,2.0,44.0,1,1.0,6.0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0.0,0.0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,525,523,3.509251,13.350405,5.868917,58.325964,8.841972,4.349597,13.422655,6.335453,52.656144,8.967193,3.276973,13.390643,5.991765,51.472567,8.924065,383050.0,186953.0,329860.0,75337.0,826.0,378239.0,233326.0,387325.0,106794.0,3272.0,998095.0,579609.0,1026804.0,253673.0,8064.0,282587.520824,280611.766021,284329.195532,279697.715188,336615.636364,3.276973,13.390643,51.472567,8.924065,5.991765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,1,1,1
1,3203,25893411,7394,7663,2.589341e+09,home,13.933333,17.783333,7.716667,8.766667,1.050000,183.0,69.0,-122.496138,37.736289,-122.450262,37.776308,drive_alone,8.033333,8.766667,16.800000,17.783333,0.983333,drive_alone,0,1,3,2589341,1,2.0,37.0,0,4.0,NaN,NaN,NaN,40.0,6.0,52.0,1,1.0,8.0,5,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,1.0,0.0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,183,69,9.487323,36.093055,15.866721,157.685268,23.904427,11.759213,36.288384,17.128008,142.356813,24.242964,14.166667,36.201839,16.198843,139.156989,24.126367,406461.0,104443.0,374588.0,31419.0,1541.0,436772.0,261837.0,420902.0,118796.0,10802.0,915147.0,599577.0,895780.0,273038.0,27471.0,227631.760137,229196.431064,227579.411547,235577.940921,220483.941379,9.487323,36.093055,157.685268,23.904427,15.866721,NaN,NaN,NaN,NaN,NaN,

In [43]:
# Create the list of individual specific variables
ind_variables = ['TOD_3to6', 'sector_mfg','finance','scitech','sector_edu_serv','sector_healthcare','sector_gov',
                'age_16less25','age_25less40','age_50less60','female','hh_inc_less75k',
                 'lessGED_GED','somebach','Assoc','lessbach',
                 'noveh','hh_size_3per','hh_size_4plusper','tenure_2',
                'total_jobs_gen_tt_CAR_AM_30','total_jobs_gen_tt_WALK_AM_30',
                'sum_residential_units_gen_tt_CAR_AM_30','sum_persons_gen_tt_CAR_AM_30']

In [10]:
# alt_varying_variables = ('gen_tt_CAR','gen_tt_WALK_TRANSIT','gen_tt_DRIVE_TRANSIT','gen_TT_BIKE','gen_tt_WALK')

In [45]:
# Specify the variables that vary across individuals **AND** 
# across some or all alternatives
alt_varying_variables = {u'travel_time': dict([(0, 'gen_tt_CAR'),
                                               (1, 'gen_tt_CAR'),
                                               (2, 'gen_tt_WALK_TRANSIT'),
                                               (3, 'gen_tt_DRIVE_TRANSIT'),
                                               (5, 'gen_tt_BIKE'),
                                               (6, 'gen_tt_WALK')]),
                        u'travel_cost': dict([(0,'gen_cost_CAR'),
                                              (1,'gen_cost_CAR'),
                                              (2,'gen_cost_WALK_TRANSIT'),
                                              (3,'gen_cost_DRIVE_TRANSIT'), 
                                              (5,'gen_cost_BIKE'),
                                              (6,'gen_cost_WALK')])}

In [46]:
# Specify the availability variables
availability_variables = {0:'DRIVE',
                          1:'SHARED',
                          2:'WALK_TRANSIT',
                          3:'DRIVE_TRANSIT',
                          5: 'BIKE',
                          6: 'WALK'}

In [47]:
# Determine the columns that will denote the
# new column of alternative ids, and the columns
# that denote the custom observation ids and the 
# choice column
new_alt_id = "mode_id"
obs_id_column = "custom_id"
choice_column = "modeHW"

In [49]:
# Perform the desired conversion
trips1 = pl.convert_wide_to_long(trips_1, 
                                           ind_variables, 
                                           alt_varying_variables, 
                                           availability_variables, 
                                           obs_id_column, 
                                           choice_column,
                                           new_alt_id_name=new_alt_id)

# Look at the first 9 rows of the long-format dataframe
trips1.head(9).T

/home/juan/anaconda3/envs/template-env/lib/python3.6/site-packages/pylogit/choice_tools.py:453: UserWarning: Note, there are 158 variables in wide_data but the inputs ind_vars, alt_specific_vars, and subset_specific_vars only account for 42 variables.
  msg_2 + msg_3.format(num_vars_accounted_for))


,0,1,2,3,4,5,6,7,8
custom_id,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,2.000000
mode_id,0.000000,1.000000,2.000000,3.000000,5.000000,6.000000,0.000000,1.000000,2.000000
modeHW,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
TOD_3to6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
sector_mfg,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
finance,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
scitech,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
sector_edu_serv,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
sector_healthcare,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
sector_gov,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [50]:
trips1.head(9).T

,0,1,2,3,4,5,6,7,8
custom_id,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,2.000000
mode_id,0.000000,1.000000,2.000000,3.000000,5.000000,6.000000,0.000000,1.000000,2.000000
modeHW,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
TOD_3to6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
sector_mfg,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
finance,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
scitech,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
sector_edu_serv,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
sector_healthcare,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
sector_gov,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [51]:
trips1["mode_id"] = pd.to_numeric(trips1["mode_id"])

In [52]:
trips1.columns

Index(['custom_id', 'mode_id', 'modeHW', 'TOD_3to6', 'sector_mfg', 'finance',
       'scitech', 'sector_edu_serv', 'sector_healthcare', 'sector_gov',
       'age_16less25', 'age_25less40', 'age_50less60', 'female',
       'hh_inc_less75k', 'lessGED_GED', 'somebach', 'Assoc', 'lessbach',
       'noveh', 'hh_size_3per', 'hh_size_4plusper', 'tenure_2',
       'total_jobs_gen_tt_CAR_AM_30', 'total_jobs_gen_tt_WALK_AM_30',
       'sum_residential_units_gen_tt_CAR_AM_30',
       'sum_persons_gen_tt_CAR_AM_30', 'travel_time', 'travel_cost'],
      dtype='object')

In [53]:
# Specify the availability variables
availability_variables = {0:'DRIVE',
                          1:'SHARED',
                          2:'WALK_TRANSIT',
                          3:'DRIVE_TRANSIT',
                          5: 'BIKE',
                          6: 'WALK'}

In [62]:
#My Nested implementation 

# Nesting structure 
nest_membership = OrderedDict()
nest_membership["Car"] = [0]
nest_membership["Share"] = [1]
nest_membership["Transit"] = [2,3]
nest_membership["Bike"] = [5]
nest_membership["Walk"] = [6]

# Specification
basic_specification = OrderedDict()
basic_names = OrderedDict()

basic_specification["intercept"] = [1,2,3,5,6]
basic_names["intercept"] = ['ASC Drive', 'ASC Walk to Transit', 'ASC Drive_Transit', 'ASC Bike', 'ASC Walk']

basic_specification["travel_time"] = [[0,1],[2,3],5,6]
basic_names["travel_time"] = ['Car TT','Transit TT','Bike TT','Walk TT']

basic_specification["travel_cost"] = [[0,1],[2,3],5,6]
basic_names["travel_cost"] = ['Car $','Transit $','Bike $','Walk $']

# basic_specification["travel_time"] = [[0,1,2,3,5,6]]
# basic_names["travel_time"] = ['Travel Time']

# basic_specification["travel_cost"] = [[0,1,2,3,5,6]]
# basic_names["travel_cost"] = ['Travel Cost']

basic_specification["total_jobs_gen_tt_CAR_AM_30"] = [0]
basic_names["total_jobs_gen_tt_CAR_AM_30"] = ['Total Jobs_30_CAR']

basic_specification["total_jobs_gen_tt_WALK_AM_30"] = [6]
basic_names["total_jobs_gen_tt_WALK_AM_30"] = ['Total Jobs_30_WALK']

basic_specification["sum_residential_units_gen_tt_CAR_AM_30"] = [0,1]
basic_names["sum_residential_units_gen_tt_CAR_AM_30"] = ['Sum Res Units_30_CAR','Sum Res Units_30_shared']

basic_specification["sum_persons_gen_tt_CAR_AM_30"] = [0,1]
basic_names["sum_persons_gen_tt_CAR_AM_30"] = ['Sum Persons_30_CAR','Sum Persons_30_shared']

basic_specification["TOD_3to6"] = [3]
basic_names["TOD_3to6"] = ['TOD_3to6_DrTran']

basic_specification["sector_mfg"] = [3]
basic_names["sector_mfg"] = ['sector_mfg_DrTran']

basic_specification["finance"] = [2,3]
basic_names["finance"] = ['finance_WTran','finance_DrTran']

basic_specification["scitech"] = [3]
basic_names["scitech"] = ['scitech_DrTran']

basic_specification["sector_edu_serv"] = [5,6]
basic_names["sector_edu_serv"] = ['sector_edu_serv_bike','sector_edu_serv_walk']

basic_specification["sector_healthcare"] = [3]
basic_names["sector_healthcare"] = ['sector_healthcare_DrTran']

basic_specification["sector_gov"] = [2]
basic_names["sector_gov"] = ['sector_gov_WTran']

basic_specification["age_16less25"] = [1]
basic_names["age_16less25"] = ['age_16less25_shared']  

basic_specification["age_25less40"] = [2,5]
basic_names["age_25less40"] = ['age_25less40_WTran','age_25less40_bike'] 

basic_specification["age_50less60"] = [1]
basic_names["age_50less60"] = ['age_50less60_shared']  

basic_specification["female"] = [1,5]
basic_names["female"] = ['female_shared','female_bike'] 

basic_specification["hh_inc_less75k"] = [1,2]
basic_names["hh_inc_less75k"] = ['hh_inc_less75k_shared','hh_inc_less75k_WTran'] 

basic_specification["lessGED_GED"] = [2]
basic_names["lessGED_GED"] = ['lessGED_GED_WTran']

basic_specification["somebach"] = [2]
basic_names["somebach"] = ['somebach_WTran']

basic_specification["Assoc"] = [2,6]
basic_names["Assoc"] = ['Assoc_WTran','Assoc_walk']

basic_specification["lessbach"] = [5]
basic_names["lessbach"] = ['lessbach_bike']

basic_specification["noveh"] = [1,2,5,6]
basic_names["noveh"] = ['noveh Shared', 'noveh Walk_Transit', 'noveh Bike', 'noveh Walk']
    
basic_specification["hh_size_3per"] = [2]
basic_names["hh_size_3per"] = ['hh_size_3per_WTran']

basic_specification["hh_size_4plusper"] = [2]
basic_names["hh_size_4plusper"] = ['hh_size_4plusper_WTran']
    
basic_specification["tenure_2"] = [2,6]
basic_names["tenure_2"] = ['rent_WTran','rent_walk']

#Runnign the model
WMCM_nested = pl.create_choice_model(data=trips1,
                                        alt_id_col=new_alt_id,
                                        obs_id_col=obs_id_column,
                                        choice_col=choice_column,
                                        specification=basic_specification,
                                        model_type= 'MNL', #"Nested Logit",
                                        names=basic_names)#,
                                        #nest_spec=nest_membership)

WMCM_nested.fit_mle(np.zeros(49))#,constrained_pos=[0])

WMCM_nested.get_statsmodels_summary()

Log-likelihood at zero: -6,572.1737
Initial Log-likelihood: -6,572.1737


/home/juan/anaconda3/envs/template-env/lib/python3.6/site-packages/scipy/optimize/_minimize.py:507: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  RuntimeWarning)


Estimation Time for Point Estimation: 0.78 seconds.
Final log-likelihood: -2,585.2569


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                      modeHW   No. Observations:                3,668
Model:             Multinomial Logit Model   Df Residuals:                    3,619
Method:                                MLE   Df Model:                           49
Date:                     Mon, 08 Jul 2019   Pseudo R-squ.:                   0.607
Time:                             15:59:08   Pseudo R-bar-squ.:               0.599
AIC:                             5,268.514   Log-Likelihood:             -2,585.257
BIC:                             5,572.676   LL-Null:                    -6,572.174
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
ASC Drive                   -3.3601      0.204    -16.498      0.000      -3.759      -2.961
ASC Walk to Transit         -3.7388      0.200    -18.722      0.000      -4.130      -3.347
ASC Drive_Transit           -4.3238      0.173    -25.033      0.000      -4.662      -3.985
ASC Bike                    -2.2573      0.229     -9.869      0.000      -2.706      -1.809
ASC Walk                    -3.0998      0.300    -10.344      0.000      -3.687      -2.512
Car TT                      -0.0107      0.004     -2.447      0.014      -0.019      -0.002
Transit TT                   0.0060      0.003      2.105      0.035       0.000       0.012
Bike TT                     -0.0249      0.011     -2.367      0.018      -0.046      -0.004
Walk TT                     -0.0151      0.005     -3.210      0.001      -0.024      -0.006
Car $                       -0.0154      0.004     -4.000      0.000      -0.023      -0.008
Transit $                   -0.0007      0.000     -6.949      0.000      -0.001      -0.001
Bike $                      -0.0253      0.033     -0.762      0.446      -0.090       0.040
Walk $                      -0.0002      0.000     -0.349      0.727      -0.001       0.001
Total Jobs_30_CAR         2.261e-06   8.43e-07      2.680      0.007    6.07e-07    3.91e-06
Total Jobs_30_WALK        1.001e-05   2.69e-06      3.720      0.000    4.73e-06    1.53e-05
Sum Res Units_30_CAR     -1.908e-05   2.13e-06     -8.941      0.000   -2.33e-05   -1.49e-05
Sum Res Units_30_shared  -1.258e-05   3.76e-06     -3.350      0.001   -1.99e-05   -5.22e-06
Sum Persons_30_CAR         5.83e-06   9.56e-07      6.100      0.000    3.96e-06     7.7e-06
Sum Persons_30_shared     4.427e-06   1.69e-06      2.621      0.009    1.12e-06    7.74e-06
TOD_3to6_DrTran             -2.2730      1.013     -2.243      0.025      -4.259      -0.287
sector_mfg_DrTran           -1.1400      0.538     -2.118      0.034      -2.195      -0.085
finance_WTran                1.0529      0.254      4.140      0.000       0.554       1.551
finance_DrTran               1.0743      0.234      4.599      0.000       0.617       1.532
scitech_DrTran               0.7121      0.186      3.823      0.000       0.347       1.077
sector_edu_serv_bike         0.8941      0.237      3.767      0.000       0.429       1.359
sector_edu_serv_walk         0.9767      0.294      3.318      0.001       0.400       1.554
sector_healthcare_DrTran    -0.5560      0.298     -1.869      0.062      -1.139       0.027
sector_gov_WTran             0.5018      0.256      1.959      0.050      -0.000       1.004
age_16less25_shared          0.5040      0.269      1.876      0.061      -0.023       1.031
age_25less40_WTran           0.4538      0.183      2.484      0.013       0.096       0.812
age_25less40_bike            0.5331      0.236      2.263      0.024       0.071       0.995
age_50less60_shared         -0.542

In [ ]:
# -3,147.161 Nesting private (car/sharing) public transit (walk and drive to transit), walk and bike
# -3,185.763 Not nesting any alternative 
# -3,147.1610 nesting walk and bike into one nest 
# -2,624.118 Emmas variables
# -2,585.257 Adding travel cost [with Emma's nesting structure for coefficients]
# -2,714.668 Travel cost and time as a alternative generic variable (1 estimation for all options) [Exclude]
# -2,585.257 Not nesting 

In [71]:
trips1.columns

Index(['custom_id', 'mode_id', 'modeHW', 'TOD_3to6', 'sector_mfg', 'finance',
       'scitech', 'sector_edu_serv', 'sector_healthcare', 'sector_gov',
       'age_16less25', 'age_25less40', 'age_50less60', 'female',
       'hh_inc_less75k', 'lessGED_GED', 'somebach', 'Assoc', 'lessbach',
       'noveh', 'hh_size_3per', 'hh_size_4plusper', 'tenure_2',
       'total_jobs_gen_tt_CAR_AM_30', 'total_jobs_gen_tt_WALK_AM_30',
       'sum_residential_units_gen_tt_CAR_AM_30',
       'sum_persons_gen_tt_CAR_AM_30', 'travel_time', 'travel_cost',
       'intercept'],
      dtype='object')

In [86]:
trips_1.columns[-20:]

Index(['gen_tt_CAR', 'gen_tt_BIKE', 'gen_tt_WALK', 'gen_tt_WALK_TRANSIT',
       'gen_tt_DRIVE_TRANSIT', 'gen_cost_BIKE', 'gen_cost_CAR',
       'gen_cost_DRIVE_TRANSIT', 'gen_cost_RIDE_HAIL',
       'gen_cost_RIDE_HAIL_POOLED', 'gen_cost_RIDE_HAIL_TRANSIT',
       'gen_cost_WALK', 'gen_cost_WALK_TRANSIT', 'DRIVE', 'SHARED',
       'WALK_TRANSIT', 'DRIVE_TRANSIT', 'BIKE', 'WALK', 'custom_id'],
      dtype='object')

In [78]:
#Adding table to orca 
orca.add_table('trips_table', trips_1, cache=True)

In [87]:
m = SmallMultinomialLogitStep()
m.name = 'mode_to_work'
m.tables = ['trips_table']
m.choice_column = 'modeHW'
m.model_expression = OrderedDict([
    ('intercept', [1,2,3,5,6]), 
    
    ('gen_tt_CAR',[0]),
    ('gen_tt_WALK_TRANSIT',[2]),
    ('gen_tt_DRIVE_TRANSIT',[3]),
    ('gen_tt_BIKE',[5]),
    ('gen_tt_WALK',[6]),
    ('gen_cost_CAR',[0]),
    ('gen_cost_WALK_TRANSIT',[2]),
    ('gen_cost_DRIVE_TRANSIT',[3]),
    ('gen_cost_BIKE',[5]),
    ('gen_cost_WALK',[6]),
    ('total_jobs_gen_tt_CAR_AM_30',[0]),
    ('total_jobs_gen_tt_WALK_AM_30',[6]),
    ('sum_residential_units_gen_tt_CAR_AM_30',[0,1]),
    ('sum_persons_gen_tt_CAR_AM_30',[0,1]),
    ('TOD_3to6',[3]),
    ('sector_mfg',[3]),
    ('finance',[2,3]),
    ('scitech',[3]),
    ('sector_edu_serv',[5,6]),
    ('sector_healthcare',[3]),
    ('sector_gov',[2]),
    ('age_16less25',[1]),
    ('age_25less40',[2,5]),
    ('age_50less60',[1]),
    ('female',[1,5]),
    ('hh_inc_less75k',[1,2]), 
    ('lessGED_GED',[2]),
    ('somebach',[2]),
    ('Assoc',[2,6]),
    ('lessbach',[5]),
    ('noveh',[1,2,5,6]),
    ('hh_size_3per',[2]),
    ('hh_size_4plusper',[2]),
    
    ('tenure_2',[2,6]),
    
])

In [88]:
m.fit()

Log-likelihood at zero: -6,572.1737
Initial Log-likelihood: -6,572.1737


/home/juan/anaconda3/envs/template-env/lib/python3.6/site-packages/scipy/optimize/_minimize.py:507: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  RuntimeWarning)


Estimation Time for Point Estimation: 0.65 seconds.
Final log-likelihood: -2,574.3822
                     Multinomial Logit Model Regression Results                    
Dep. Variable:                     _chosen   No. Observations:                3,668
Model:             Multinomial Logit Model   Df Residuals:                    3,617
Method:                                MLE   Df Model:                           51
Date:                     Mon, 08 Jul 2019   Pseudo R-squ.:                   0.608
Time:                             16:48:23   Pseudo R-bar-squ.:               0.601
AIC:                             5,250.764   Log-Likelihood:             -2,574.382
BIC:                             5,567.342   LL-Null:                    -6,572.174
                                               coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
intercept_1             

In [17]:
# Specify the nesting values
nest_membership = OrderedDict()
nest_membership["car"] = [0,1]
nest_membership["transit"] = [2,3]
nest_membership["active"] = [5,6]

In [95]:
# Create the model's specification dictionary and variable names dictionary
# NOTE: - Keys should be variables within the long format dataframe.
#         The sole exception to this is the "intercept" key.
#       - For the specification dictionary, the values should be lists
#         or lists of lists. Within a list, or within the inner-most
#         list should be the alternative ID's of the alternative whose
#         utility specification the explanatory variable is entering.

example_specification = OrderedDict()
example_names = OrderedDict()

# Note that 1 is the id for the Train and 3 is the id for the Car.
# The next two lines are placing alternative specific constants in
# the utility equations for the Train and for the Car. The order
# in which these variables are placed is chosen so the summary
# dataframe which is returned will match that shown in the HTML
# file of the python biogeme example.
example_specification["intercept"] = [0,3,5]
example_names["intercept"] = ['ASC Car', 'ASC Drive_Transit', 'ASC Bike']

# Note that the names used below are simply for consistency with
# the coefficient names given in the Python Biogeme example.
# example_specification["travel_cost_hundredth"] = [[1, 2, 3]]
# example_names["travel_cost_hundredth"] = ['B_COST']

example_specification["travel_time"] = [[0,1],[2,3],5,6]
example_names["travel_time"] = ['Car TT','Transit TT','Bike TT','Walk TT']

example_specification["total_jobs_gen_tt_CAR_AM_30"] = [0]
example_names["total_jobs_gen_tt_CAR_AM_30"] = ['Total Jobs_30_CAR']

example_specification["total_jobs_gen_tt_WALK_AM_30"] = [6]
example_names["total_jobs_gen_tt_WALK_AM_30"] = ['Total Jobs_30_WALK']

example_specification["sum_residential_units_gen_tt_CAR_AM_30"] = [0,1]
example_names["sum_residential_units_gen_tt_CAR_AM_30"] = ['Sum Res Units_30_CAR','Sum Res Units_30_shared']

example_specification["sum_persons_gen_tt_CAR_AM_30"] = [0,1]
example_names["sum_persons_gen_tt_CAR_AM_30"] = ['Sum Persons_30_CAR','Sum Persons_30_shared']


example_specification["TOD_3to6"] = [3]
example_names["TOD_3to6"] = ['TOD_3to6_DrTran']

example_specification["sector_mfg"] = [3]
example_names["sector_mfg"] = ['sector_mfg_DrTran']

example_specification["finance"] = [2,3]
example_names["finance"] = ['finance_WTran','finance_DrTran']

example_specification["scitech"] = [3]
example_names["scitech"] = ['scitech_DrTran']

example_specification["sector_edu_serv"] = [5,6]
example_names["sector_edu_serv"] = ['sector_edu_serv_bike','sector_edu_serv_walk']

example_specification["sector_healthcare"] = [3]
example_names["sector_healthcare"] = ['sector_healthcare_DrTran']

example_specification["sector_gov"] = [2]
example_names["sector_gov"] = ['sector_gov_WTran']

example_specification["age_16less25"] = [1]
example_names["age_16less25"] = ['age_16less25_shared']  

example_specification["age_25less40"] = [2,5]
example_names["age_25less40"] = ['age_25less40_WTran','age_25less40_bike'] 

example_specification["age_50less60"] = [1]
example_names["age_50less60"] = ['age_50less60_shared']  

example_specification["female"] = [1,5]
example_names["female"] = ['female_shared','female_bike'] 

example_specification["hh_inc_less75k"] = [1,2]
example_names["hh_inc_less75k"] = ['hh_inc_less75k_shared','hh_inc_less75k_WTran'] 

example_specification["lessGED_GED"] = [2]
example_names["lessGED_GED"] = ['lessGED_GED_WTran']

example_specification["somebach"] = [2]
example_names["somebach"] = ['somebach_WTran']

example_specification["Assoc"] = [2,6]
example_names["Assoc"] = ['Assoc_WTran','Assoc_walk']

example_specification["lessbach"] = [5]
example_names["lessbach"] = ['lessbach_bike']

example_specification["noveh"] = [1,2,5,6]
example_names["noveh"] = ['noveh Shared', 'noveh Walk_Transit', 'noveh Bike', 'noveh Walk']
    
example_specification["hh_size_3per"] = [2]
example_names["hh_size_3per"] = ['hh_size_3per_WTran']

example_specification["hh_size_4plusper"] = [2]
example_names["hh_size_4plusper"] = ['hh_size_4plusper_WTran']
    
example_specification["tenure_2"] = [2,6]
example_names["tenure_2"] = ['rent_WTran','rent_walk']

In [81]:
# Define a function that calculates the "logit" transformation of values
# between 0.0 and 1.0.
def logit(x):
    """
    Parameters
    ----------
    x : int, float, or 1D ndarray.
        If an array, all elements should be ints or floats. All
        elements should be between zero and one, exclusive of 1.0.

    Returns
    -------
    The logit of x:  `np.log(x / (1.0 - x))`.
    """
    return np.log(x/(1.0 - x))

In [97]:
# Provide the module with the needed input arguments to create
# an instance of the MNL model class
example_nested = pl.create_choice_model(data=trips1,
                                        alt_id_col=new_alt_id,
                                        obs_id_col=obs_id_column,
                                        choice_col=choice_column,
                                        specification=example_specification,
                                        model_type="Nested Logit",
                                        names=example_names,
                                        nest_spec=nest_membership)

# # Specify the initial nesting parameter values
# # Note: This should be in terms of the reparameterized values used
# # by PyLogit.

# # Note: The '40' corresponds to scale parameter that is numerically
# # indistinguishable from 1.0

# # Note: 2.05 is the scale parameter that is estimated by PythonBiogeme
# # so we invert it, then take the logit of this inverse to get the
# # corresponding starting value to be used by PyLogit.
# # Note the first value corresponds to the first nest in 'nest_spec'
# # and the second value corresponds to the second nest in 'nest_spec'.
# init_nests = np.array([40, logit(2.05**-1)])

# # Specify the initial index coefficients used by PythonBiogeme
# init_coefs = np.array([-0.167, -0.512, -0.899, -0.857])

# # Create a single array of the initial values
# init_values = np.concatenate((init_nests, init_coefs), axis=0)

# Start the model estimation from the pythonbiogeme initial values
# Note that the first value, in the initial values, is constrained
# to remain constant through the estimation process. This is because
# the first nest in nest_spec is a 'degenerate' nest with only one
# alternative, and the nest parameter of degenerate nests is not
# identified.
example_nested.fit_mle(np.zeros(46))

Log-likelihood at zero: -6,572.1737
Initial Log-likelihood: -6,572.1737


/home/emma/anaconda/envs/ual-model/lib/python3.6/site-packages/scipy/optimize/_minimize.py:506: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  RuntimeWarning)


Estimation Time for Point Estimation: 2.44 minutes.
Final log-likelihood: -2,624.3745


/home/emma/anaconda/envs/ual-model/lib/python3.6/site-packages/pylogit/base_multinomial_cm_v2.py:1259: RuntimeWarning: invalid value encountered in sqrt
  self._store_inferential_results(np.sqrt(np.diag(self.robust_cov)),
/home/emma/anaconda/envs/ual-model/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/home/emma/anaconda/envs/ual-model/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/home/emma/anaconda/envs/ual-model/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [98]:
# Look at the estimated coefficients and goodness-of-fit statistics
example_nested.get_statsmodels_summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                    Nested Logit Model Regression Results                     
==============================================================================
Dep. Variable:                 modeHW   No. Observations:                3,668
Model:             Nested Logit Model   Df Residuals:                    3,622
Method:                           MLE   Df Model:                           46
Date:                Thu, 25 Apr 2019   Pseudo R-squ.:                   0.601
Time:                        18:17:13   Pseudo R-bar-squ.:               0.594
AIC:                        5,340.749   Log-Likelihood:             -2,624.375
BIC:                        5,626.290   LL-Null:                    -6,572.174
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
car                          4.1928      5.013      0.836      0.403      -5.632      14.017
transit                     -2.2526      0.494     -4.557      0.000      -3.221      -1.284
active                      31.2470   9.91e+12   3.15e-12      1.000   -1.94e+13    1.94e+13
ASC Car                      3.3182      0.152     21.798      0.000       3.020       3.617
ASC Drive_Transit           -0.0732      0.039     -1.877      0.061      -0.150       0.003
ASC Bike                     0.9199      0.219      4.206      0.000       0.491       1.349
Car TT                      -0.0248      0.003     -7.493      0.000      -0.031      -0.018
Transit TT                  -0.0046      0.002     -2.395      0.017      -0.008      -0.001
Bike TT                     -0.0326      0.003    -10.911      0.000      -0.038      -0.027
Walk TT                     -0.0156      0.002     -7.077      0.000      -0.020      -0.011
Total Jobs_30_CAR         2.131e-06   8.36e-07      2.548      0.011    4.92e-07    3.77e-06
Total Jobs_30_WALK        9.798e-06   2.91e-06      3.367      0.001    4.09e-06    1.55e-05
Sum Res Units_30_CAR     -2.039e-05   2.13e-06     -9.560      0.000   -2.46e-05   -1.62e-05
Sum Res Units_30_shared   -1.42e-05   3.73e-06     -3.810      0.000   -2.15e-05   -6.89e-06
Sum Persons_30_CAR        6.381e-06   9.68e-07      6.593      0.000    4.48e-06    8.28e-06
Sum Persons_30_shared     5.051e-06   1.72e-06      2.941      0.003    1.69e-06    8.42e-06
TOD_3to6_DrTran             -0.1530      0.130     -1.181      0.238      -0.407       0.101
sector_mfg_DrTran           -0.0874      0.096     -0.909      0.363      -0.276       0.101
finance_WTran                0.9884      0.181      5.447      0.000       0.633       1.344
finance_DrTran               0.9866      0.180      5.489      0.000       0.634       1.339
scitech_DrTran               0.0363      0.035      1.042      0.297      -0.032       0.105
sector_edu_serv_bike         0.9175      0.233      3.937      0.000       0.461       1.374
sector_edu_serv_walk         1.0350      0.254      4.079      0.000       0.538       1.532
sector_healthcare_DrTran    -0.0337      0.043     -0.781      0.435      -0.118       0.051
sector_gov_WTran             0.0236      0.043      0.548      0.583      -0.061       0.108
age_16less25_shared          0.5387      0.267      2.018      0.044       0.016       1.062
age_25less40_WTran           0.0253      0.031      0.809      0.418      -0.036       0.087
age_25less40_bike            0.4445      0.235      1.891      0.059      -0.016       0.905
age_50less60_shared         -0.5111      0.197     -2.598      0.009      -0.897      -0.125
female_shared                0.7823      0.152      5.138      0.000       0.484       1.081
female_bike                 -1.4555      0.274     -5.313      0.000      -1.992      -0.919
hh_inc_less75k_shared        0.2137      0.171      1.249      0.211      -0.12

In [99]:
validate = pd.read_csv('/home/emma/activitysynth/activitysynth/data/HWtrips_validate_042519.csv')

In [100]:
validate['DRIVE'] = 1
validate['SHARED'] = 1
validate['WALK_TRANSIT'] = 1
validate['DRIVE_TRANSIT'] = 1
validate['BIKE'] = 1
validate['WALK'] = 1

In [102]:
validate["custom_id"] = np.arange(validate.shape[0], dtype=int) + 1

In [103]:
# Perform the desired conversion
validate = pl.convert_wide_to_long(validate, 
                                           ind_variables, 
                                           alt_varying_variables, 
                                           availability_variables, 
                                           obs_id_column, 
                                           choice_column,
                                           new_alt_id_name=new_alt_id)

# Look at the first 9 rows of the long-format dataframe
validate.head(9).T

/home/emma/anaconda/envs/ual-model/lib/python3.6/site-packages/pylogit/choice_tools.py:453: UserWarning: Note, there are 151 variables in wide_data but the inputs ind_vars, alt_specific_vars, and subset_specific_vars only account for 37 variables.
  msg_2 + msg_3.format(num_vars_accounted_for))


,0,1,2,3,4,5,6,7,8
custom_id,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,2.000000
mode_id,0.000000,1.000000,2.000000,3.000000,5.000000,6.000000,0.000000,1.000000,2.000000
modeHW,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
TOD_3to6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
sector_mfg,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
finance,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
scitech,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
sector_edu_serv,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
sector_healthcare,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
sector_gov,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [104]:
prediction_array = example_nested.predict(validate)

In [105]:
prediction_array

array([0.85815185, 0.07247097, 0.03006816, ..., 0.00954862, 0.03839138,
       0.06635421])

In [107]:
# total_weights = np.sum(data_wide.loc[:9999, 'weights'])
# weights = new_data["weights"] / total_weights

alternative_names = ["drive", "shared ride", 
                     "walk transit", "drive transit",
                    "bike","walk"]
num_alternatives = validate["mode_id"].unique().size
market_shares = np.empty(num_alternatives)
for i in range(1, 7):
    filter_condition = (validate["mode_id"] == i).values
    num_obs_in_condition = filter_condition.sum()
#     current_weights = weights[filter_condition]
#     assert current_weights.size == num_obs_in_condition
    current_alternative_share =\
        prediction_array[filter_condition].dot(current_weights)
    market_shares[i - 1] = current_alternative_share
    
pd.Series(market_shares, index=alternative_names)

TypeError: Required argument 'b' (pos 1) not found

In [25]:
import random 

In [54]:
from urbansim_templates.models import SmallMultinomialLogitStep
from collections import OrderedDict

In [86]:
df_example = pd.DataFrame({'obs_id': [1,1,1,2,2,2,3,3,3,4,4,4], 
                           'alt_id': [1,2,3,1,2,3,1,2,3,1,2,3],
                           'chosen': [1,0,0,0,1,0,0,1,0,1,0,0], 
                           'tt':random.sample(range(12), 12), 
                           'tc':random.sample(range(12), 12),
                           'gender': [1,1,1,0,0,0,1,1,1,0,0,0]})

orca.add_table('example', df_example)

In [89]:
df_example_wide = pd.DataFrame({'obs_id': [1,2,3,4], 'chosen': [1,2,2,1], 
                                'tt_1':[2,11,9,3], 'tt_2':[10,6,1,7], 'tt_3':[5,0,8,4],
                                'tc_1':[2,11,10,5], 'tc_2':[6,9,7,1], 'tc_3': [4,0,8,3],
                                'gender': [1,0,1,0]})
orca.add_table('example_wide', df_example_wide)

In [109]:
model_expresion = OrderedDict({'tt_1':[[1]], 'tc_2':[1], 'gender': [2,3]})
model_labels = OrderedDict({'tt_1':['tt'], 'tc_2':['tc'], 'gender': ['gender_2','gender_3']})
model = SmallMultinomialLogitStep(tables = 'example_wide', 
                          model_expression = model_expresion, 
                          model_labels = model_labels, 
                          choice_column='chosen',
                          initial_coefs = [0,0,0,0,0,0])

In [108]:
model.fit()

Log-likelihood at zero: -4.3944
Initial Log-likelihood: -4.3944
Estimation Time for Point Estimation: 0.03 seconds.
Final log-likelihood: -3.0543
                     Multinomial Logit Model Regression Results                    
Dep. Variable:                     _chosen   No. Observations:                    4
Model:             Multinomial Logit Model   Df Residuals:                        0
Method:                                MLE   Df Model:                            4
Date:                     Wed, 07 Aug 2019   Pseudo R-squ.:                   0.305
Time:                             10:43:29   Pseudo R-bar-squ.:              -0.605
AIC:                                14.109   Log-Likelihood:                 -3.054
BIC:                                11.654   LL-Null:                        -4.394
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
tt            -0.3375   

In [87]:
df_example

,obs_id,alt_id,chosen,tt,tc,gender
0,1,1,1,2,2,1
1,1,2,0,10,6,1
2,1,3,0,5,4,1
3,2,1,0,11,11,0
4,2,2,1,6,9,0
5,2,3,0,0,0,0
6,3,1,0,9,10,1
7,3,2,1,1,7,1
8,3,3,0,8,8,1
9,4,1,1,3,5,0


In [78]:
model_expresion = OrderedDict({'tt':[[1,2,3]], 'tc':[[1,2,3]], 'gender': [2,3]})
model_labels = OrderedDict({'tt':['tt'], 'tc':['tc'], 'gender': ['gender_2','gender_3']})

In [79]:
model = SmallMultinomialLogitStep(tables = 'example', 
                          model_expression = model_expresion, 
                          model_labels = model_labels, 
                          choice_column='chosen',
                          initial_coefs = [0,0,0,0,0,0])

In [80]:
model.fit()

Log-likelihood at zero: -4.3944
Initial Log-likelihood: -4.3944
Estimation Time for Point Estimation: 0.03 seconds.
Final log-likelihood: -4.3944


/home/juan/anaconda3/envs/template-env/lib/python3.6/site-packages/scipy/optimize/_minimize.py:507: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  RuntimeWarning)


LinAlgError: singular matrix

In [82]:
trips_1

,Unnamed: 0,HHPER,index,Unnamed: 0.1,HHPERTRIP,origin,home_dwell,home_ST,HW_trip_ST,HW_trip_ET,HW_TT,Hzone,Wzone,Hxcord,Hycord,Wxcord,Wycord,HW_mode,work_dwell,work_ST,WH_trip_ST,WH_trip_ET,WH_TT,WH_mode,modeHW,TOD,dwell_work,SAMPN,PERNO,GEND,AGE,HISP,RACE1,RACE2,RACE3,RACE4,hours,EDUCA,INDUS,HHVEH,OWN,INCOM,HHSIZ,minority,black,native,asian,PI,hh_inc_less10k,hh_inc_10kless25k,hh_inc_25kless35k,hh_inc_35kless50k,hh_inc_50kless75k,hh_inc_100kless150k,hh_inc_150kless200k,hh_inc_200kless250k,hh_inc_less50k,hh_inc_less75k,hh_inc_75kless100k,hh_inc_150kplus,hh_inc_150kless250k,hh_inc_250kplus,lessGED,GED,somebach,Assoc,Bach,lessGED_GED,lessbach,age_16less25,age_25less40,age_40less50,age_50less60,female,tenure_2,noveh,1veh,2veh,3plusveh,hh_size_1per,hh_size_2per,hh_size_3per,hh_size_4plusper,sector_constr,sector_mfg,sector_retail,sector_transport,info,finance,scitech,sector_edu_serv,sector_healthcare,sector_oth_serv,sector_gov,TOD_3to6,TOD_6to9,TOD_9to1530,TOD_1530to1830,TOD_1830up,TOD_9up,TOD_1530up,from_zone_id,to_zone_id,gen_tt_CAR_AM,gen_tt_BIKE_AM,gen_tt_DRIVE_TRANSIT_AM,gen_tt_WALK_AM,gen_tt_WALK_TRANSIT_AM,gen_tt_CAR_PM,gen_tt_BIKE_PM,gen_tt_DRIVE_TRANSIT_PM,gen_tt_WALK_PM,gen_tt_WALK_TRANSIT_PM,gen_tt_CAR_OffPeak,gen_tt_BIKE_OffPeak,gen_tt_DRIVE_TRANSIT_OffPeak,gen_tt_WALK_OffPeak,gen_tt_WALK_TRANSIT_OffPeak,total_jobs_gen_tt_CAR_AM_30,total_jobs_gen_tt_WALK_TRANSIT_AM_30,total_jobs_gen_tt_DRIVE_TRANSIT_AM_30,total_jobs_gen_tt_BIKE_AM_30,total_jobs_gen_tt_WALK_AM_30,sum_residential_units_gen_tt_CAR_AM_30,sum_residential_units_gen_tt_WALK_TRANSIT_AM_30,sum_residential_units_gen_tt_DRIVE_TRANSIT_AM_30,sum_residential_units_gen_tt_BIKE_AM_30,sum_residential_units_gen_tt_WALK_AM_30,sum_persons_gen_tt_CAR_AM_30,sum_persons_gen_tt_WALK_TRANSIT_AM_30,sum_persons_gen_tt_DRIVE_TRANSIT_AM_30,sum_persons_gen_tt_BIKE_AM_30,sum_persons_gen_tt_WALK_AM_30,avg_income_gen_tt_CAR_AM_30,avg_income_gen_tt_WALK_TRANSIT_AM_30,avg_income_gen_tt_DRIVE_TRANSIT_AM_30,avg_income_gen_tt_BIKE_AM_30,avg_income_gen_tt_WALK_AM_30,gen_tt_CAR,gen_tt_BIKE,gen_tt_WALK,gen_tt_WALK_TRANSIT,gen_tt_DRIVE_TRANSIT,gen_cost_BIKE,gen_cost_CAR,gen_cost_DRIVE_TRANSIT,gen_cost_RIDE_HAIL,gen_cost_RIDE_HAIL_POOLED,gen_cost_RIDE_HAIL_TRANSIT,gen_cost_WALK,gen_cost_WALK_TRANSIT,DRIVE,SHARED,WALK_TRANSIT,DRIVE_TRANSIT,BIKE,WALK
0,3992,30202193,9223,9547,3.020219e+09,home,14.583333,7.166667,21.750000,22.000000,0.250000,525.0,523.0,-121.896879,37.280818,-121.918797,37.264669,drive_alone,9.000000,22.000000,7.000000,7.166667,0.166667,drive_alone,0,4,4,3020219,3,1.0,21.0,0,1.0,NaN,NaN,NaN,30.0,2.0,44.0,1,1.0,6.0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0.0,0.0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,525,523,3.509251,13.350405,5.868917,58.325964,8.841972,4.349597,13.422655,6.335453,52.656144,8.967193,3.276973,13.390643,5.991765,51.472567,8.924065,383050.0,186953.0,329860.0,75337.0,826.0,378239.0,233326.0,387325.0,106794.0,3272.0,998095.0,579609.0,1026804.0,253673.0,8064.0,282587.520824,280611.766021,284329.195532,279697.715188,336615.636364,3.276973,13.390643,51.472567,8.924065,5.991765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,1,1,1
1,3203,25893411,7394,7663,2.589341e+09,home,13.933333,17.783333,7.716667,8.766667,1.050000,183.0,69.0,-122.496138,37.736289,-122.450262,37.776308,drive_alone,8.033333,8.766667,16.800000,17.783333,0.983333,drive_alone,0,1,3,2589341,1,2.0,37.0,0,4.0,NaN,NaN,NaN,40.0,6.0,52.0,1,1.0,8.0,5,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,1.0,0.0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,183,69,9.487323,36.093055,15.866721,157.685268,23.904427,11.759213,36.288384,17.128008,142.356813,24.242964,14.166667,36.201839,16.198843,139.156989,24.126367,406461.0,104443.0,374588.0,31419.0,1541.0,436772.0,261837.0,420902.0,118796.0,10802.0,915147.0,599577.0,895780.0,273038.0,27471.0,227631.760137,229196.431064,227579.411547,235577.940921,220483.941379,9.487323,36.093055,157.685268,23.904427,15.866721,NaN,NaN,NaN,NaN,NaN,

In [81]:
trips_1
orca.add_table('trips', trips_1)

In [84]:
model_expresion = OrderedDict([('intercept', [0,1,3,5,6])])
# model_labels = OrderedDict({'tt':['tt'], 'tc':['tc'], 'gender': ['gender_2','gender_3']})


model = SmallMultinomialLogitStep(tables = 'trips', 
                          model_expression = model_expresion, 
#                           model_labels = model_labels, 
                          choice_column='modeHW',)
#                           initial_coefs = [0,0,0,0,0,0])

In [85]:
model.fit()

Log-likelihood at zero: -5,478.5267
Initial Log-likelihood: -5,478.5267


/home/juan/anaconda3/envs/template-env/lib/python3.6/site-packages/scipy/optimize/_minimize.py:507: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  RuntimeWarning)


Estimation Time for Point Estimation: 0.22 seconds.
Final log-likelihood: -2,681.2229
                     Multinomial Logit Model Regression Results                    
Dep. Variable:                     _chosen   No. Observations:                3,668
Model:             Multinomial Logit Model   Df Residuals:                    3,663
Method:                                MLE   Df Model:                            5
Date:                     Tue, 06 Aug 2019   Pseudo R-squ.:                   0.511
Time:                             16:17:52   Pseudo R-bar-squ.:               0.510
AIC:                             5,372.446   Log-Likelihood:             -2,681.223
BIC:                             5,403.483   LL-Null:                    -5,478.527
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
intercept_0   -96.0338   1.81e+05     -0.001      1.000   -3.55e+05    3.55e+05
in